In [1]:
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [21]:
data, temp = [],[]
gests = os.listdir('dataset')
for gest in gests:
    data = []
    print(gest)
    f = open('dataset/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        '''
        D = []
        x = f[i][0]
        y = f[i][1]
        rs_x = 1 - x
        rs_y = 1 - y
        for j in range(0, len(f[i]), 2):
            D.append(f[i][j] + rs_x)
            D.append(f[i][j + 1] + rs_y)
        '''
        data.append(f[i])
    temp.append(data)
data = np.array(temp)

rkulak.txt
r_arm0.txt
leftladon.txt
l_arm0.txt
rladon.txt
leftkulak.txt


/var/folders/05/318hv04n4zg2jytwsxm19nsw0000gn/T/ipykernel_68134/199723171.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(temp)


In [4]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

956
575
577
557
1375
1080


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 6)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [6]:
net = Net()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

In [8]:
for epoch in range(15):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss/len(dataloader))
print('Finished Training')

0.9740398391149938
0.310719003318809
0.18740016776137053
0.13265131874359212
0.1095550089579774
0.09456706345517887
0.08226291497194324
0.07559476777532836
0.08160432940421743
0.0608135043643415
0.06501565272337757
0.06792720472658402
0.055780571601644624
0.0586036681481346
0.053406078839907424
Finished Training


In [9]:
kk=torch.tensor([0.34132900834083557, 0.7993544340133667, 0.27319902181625366, 0.7789114117622375, 0.22405318915843964, 0.7397434711456299, 0.18417870998382568, 0.7060592174530029, 0.14576251804828644, 0.6867378950119019, 0.22664082050323486, 0.5892890095710754, 0.15461356937885284, 0.6505651473999023, 0.1522427797317505, 0.7229896783828735, 0.16590969264507294, 0.7556616067886353, 0.2659417986869812, 0.583932638168335, 0.17748120427131653, 0.6850151419639587, 0.19443784654140472, 0.7680878043174744, 0.21849800646305084, 0.7915521860122681, 0.3038718104362488, 0.6057591438293457, 0.21920517086982727, 0.714357852935791, 0.23477542400360107, 0.7916398644447327, 0.25649726390838623, 0.8051833510398865, 0.3381839394569397, 0.6458810567855835, 0.27132633328437805, 0.7355436086654663, 0.27562248706817627, 0.7925152778625488, 0.2939406931400299, 0.8016353249549866])

In [20]:
ls=0
for samples, labels in dataloader:
    pred = net(samples)
    v,i = torch.max(pred, 1)
    ls+=torch.sum(abs(labels - i)).item()

print((len(dataset)-ls)/len(dataset))


0.9740234375


In [11]:
v,i = torch.max(pred, 1)

In [12]:
i

tensor([0, 2, 0, 0, 0, 4, 5, 4, 4, 4, 4, 5, 5, 3, 4, 3])

In [13]:
torch.sum(labels - i)

tensor(0)